In [9]:
import pandas as pd
import sqlalchemy as sa
import urllib

In [10]:
params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()


In [11]:
df_drug_ids = pd.read_sql(sa.text('SELECT $node_id, drugbank_id FROM drug_nodes'), cnxn)
df_drug_ids

,$node_id_C59E823CFB2442BFB04D7FE55B42561F,drugbank_id
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00002
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00003
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00004
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00005
...,...,...
15230,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17382
15231,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17383
15232,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17384
15233,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17385


In [12]:
df_gene_ids = pd.read_sql(sa.text('SELECT $node_id, uniprotkb_id FROM gene_nodes'), cnxn)
df_gene_ids

,$node_id_70F4A652A18D4EAA817A05B8307F4E47,uniprotkb_id
0,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q8IX94
1,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",S4R3F8
2,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
3,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
4,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",P62805
...,...,...
44827,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44828,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44829,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",Q9Y262
44830,"{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",A0A096LPK9


In [13]:
drug_gene_df = pd.read_excel("../parser/temp/drug_target_edges.xlsx")
drug_gene_df = drug_gene_df[['drugbank-id','uniprotkb-id','action']]
drug_gene_df = drug_gene_df.dropna()
drug_gene_df.columns = 'drugbank_id','uniprotkb_id','action'
drug_gene_df

,drugbank_id,uniprotkb_id,action
0,DB00001,P00734,inhibitor
1,DB00002,P00533,binder
2,DB00002,O75015,binder
3,DB00002,P02745,binder
4,DB00002,P02746,binder
...,...,...,...
30803,DB17083,Q9NPD5,inhibitor
30804,DB17083,Q8TCC7,substrate
30805,DB17083,Q8TCC7,inhibitor
30806,DB17083,Q9UNQ0,substrate


In [14]:
df = pd.merge(drug_gene_df, df_drug_ids, on='drugbank_id', how='left')
df = pd.merge(df, df_gene_ids, on='uniprotkb_id', how='left')
df = df.dropna()
df.columns = ['drugbank_id', 'uniprotkb_id', 'action','from_id','to_id']
df = df[['from_id','to_id','drugbank_id', 'uniprotkb_id', 'action']]
df

,from_id,to_id,drugbank_id,uniprotkb_id,action
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB00001,P00734,inhibitor
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB00002,P00533,binder
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB00002,P00533,binder
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB00002,P00533,binder
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB00002,P00533,binder
...,...,...,...,...,...
52967,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB17083,Q9UNQ0,substrate
52968,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB17083,Q9UNQ0,substrate
52969,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB17083,Q4U2R8,inhibitor
52970,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""gene_no...",DB17083,Q4U2R8,inhibitor


In [15]:
df.to_csv("./ingest/drug_gene_edges_id.csv", sep="\t", index=False)